In [1]:
save.dir <- './figs/figs_feb_26/'
fs <-  4 
fs.small <- 3

#### computing diversity based on KL divergence of observed counts from expected counts based on cell subgroup proportions

In [2]:
find.div <- function(counts, meta) {
    sum.counts <- sum(counts)
    
    ## expected counts for each subgroup based on proportion of cells in that subgroup
    expected.props <- table(meta)/length(meta)
    
    ## observed counts for each subgroup
    observed.props <- sapply(names(expected.props), function(curr.group) {
        curr.cells <- intersect(colnames(counts), names(meta[meta==curr.group]))
        sum(counts[,curr.cells])/sum.counts
    })
    
    kl <- sum(observed.props * log(observed.props/expected.props))
    
    return(kl)
    
}

## DATA

### merfish

In [3]:
merfish.panels <- readRDS('./merfish_mousebrain/merfish_norm_data.rds')
merfish.full.counts <- merfish.panels$all.norms$nonorm
merfish.skewed.norms <- lapply(merfish.panels$skewed.norms, function(x) { x[['nonorm']]})
merfish.counts <- merfish.skewed.norms
merfish.counts[['full']] <- merfish.full.counts

merfish.meta <- merfish.panels$meta[colnames(merfish.full.counts),'reg.coarse']
names(merfish.meta) <- colnames(merfish.full.counts)

### starmap

In [4]:
starmap.panels <- readRDS('./STARmap/starmap_norm_data.rds')
starmap.full.counts <- starmap.panels$all.norms$nonorm
starmap.skewed.norms <- lapply(starmap.panels$skewed.norms, function(x) { x[['nonorm']]})
starmap.counts <- starmap.skewed.norms
starmap.counts[['full']] <- starmap.full.counts

starmap.meta <- starmap.panels$meta[colnames(starmap.full.counts),'region']
names(starmap.meta) <- colnames(starmap.full.counts)

### seqFISH

In [5]:
seqfish.panels <- readRDS('./seqFISH_kidney/seqfish_norm_data.rds')
seqfish.full.counts <- seqfish.panels$all.norms$nonorm
seqfish.skewed.norms <- lapply(seqfish.panels$skewed.norms, function(x) { x[['nonorm']]})
seqfish.counts <- seqfish.skewed.norms
seqfish.counts[['full']] <- seqfish.full.counts

seqfish.meta <- seqfish.panels$meta[colnames(seqfish.full.counts),'region']
names(seqfish.meta) <- colnames(seqfish.full.counts)

### cosmx

In [6]:
cosmx.panels <- readRDS('./CosMx_liver/cosmx_norm_data.rds')
cosmx.full.counts <- cosmx.panels$all.norms$nonorm
cosmx.skewed.norms <- lapply(cosmx.panels$skewed.norms, function(x) { x[['nonorm']]})
cosmx.counts <- cosmx.skewed.norms
cosmx.counts[['full']] <- cosmx.full.counts

cosmx.meta <- cosmx.panels$meta[colnames(cosmx.full.counts),'region']
names(cosmx.meta) <- colnames(cosmx.full.counts)

### xenium

In [7]:
xenium.panels <- readRDS('./xenium/xenium_norm_data.rds')
xenium.full.counts <- xenium.panels$all.norms$nonorm
xenium.skewed.norms <- lapply(xenium.panels$skewed.norms, function(x) { x[['nonorm']]})
xenium.counts <- xenium.skewed.norms
xenium.counts[['full']] <- xenium.full.counts

xenium.meta <- xenium.panels$meta[colnames(xenium.full.counts),'region']
names(xenium.meta) <- colnames(xenium.full.counts)

In [8]:
datasets <- list(merfish = list(counts = merfish.counts, meta = merfish.meta), 
                 starmap = list(counts = starmap.counts, meta = starmap.meta),
                 seqfish = list(counts = seqfish.counts, meta = seqfish.meta),
                 cosmx = list(counts = cosmx.counts, meta = cosmx.meta),
                 xenium = list(counts = xenium.counts, meta = xenium.meta))

In [26]:
names(datasets)
lapply(datasets, function(x) { dim(x[[1]][[1]]) })

[1] "merfish" "starmap" "seqfish" "cosmx"   "xenium"

$merfish
[1]   100 10753

$starmap
[1]  100 5558

$seqfish
[1]     60 250771

$cosmx
[1]   200 93503

$xenium
[1]    80 63127

[1]   100 10753

## SKEW

In [9]:
divs <- lapply(datasets, function(curr.data) {
    lapply(curr.data$counts, function(curr.counts) {
        curr.meta <- curr.data$meta[colnames(curr.counts)]
        find.div(curr.counts, curr.meta)
    })
})

In [11]:
# saveRDS(divs, file = 'panel_divs.RDS')

## single cell sim div

In [23]:
sc.sim.deprop <- readRDS('./sc_pbmc/scpbmc_DEprop_countssubsets.RDS')
sc.meta <- readRDS('./sc_pbmc/scpbmc_celltypes.RDS')
sc.sim.panelsize <- readRDS('./sc_pbmc/scpbmc_sizeSim_countssubsets.RDS')

In [24]:
sc.deprop.divs <- lapply(sc.sim.deprop, function(curr.gs.prop) {
    lapply(curr.gs.prop, function(curr.counts) {
        curr.meta <- sc.meta[colnames(curr.counts)]
        find.div(curr.counts, curr.meta)
    })
})
sc.gssize.divs <- lapply(sc.sim.panelsize, function(curr.gs.size) {
    lapply(curr.gs.size, function(curr.counts) {
        curr.meta <- sc.meta[colnames(curr.counts)]
        find.div(curr.counts, curr.meta)
    })
})

## imSRT panel skew

In [41]:
dir.create(paste0(save.dir,'panel_div'))

In [56]:
gs.names <- c('Fiber tracts', 'Ventricles', 'Habenula', 'Dentate gyrus', NA)
panel.div <- unlist(divs$merfish)
panel.div.names <- c(gs.names[1:4], 'Random', 'Original')
options(repr.plot.width=6, repr.plot.height=7)
pdf(paste0(save.dir, 'panel_div/merfish.pdf'),  
    width = 8, height = 9)
par(mfrow = c(1,1), mar = c(17.1, 10, 3.1, 2.1))
xtcks <- c(0:length(panel.div)+1)
x <- seq(length(panel.div))
y <- panel.div
xl <- c(0,max(x)+1)
yl <- c(0,0.08)
sp <- c(1.5, rep(1,length(panel.div)-1))

barplot(y, xlim = xl, ylim = yl, width = 0.5, space = sp,
    xaxt = "n", yaxt = "n", xaxs = 'i', yaxs = 'i',
    ylab = '', xlab = '')
axis(1, at = c(0,xtcks), cex.axis=2, labels = FALSE)
axis(2, at = seq(from = 0, to = ceiling(max(yl)), by = 0.02), cex.axis=3, las = 2)
text(x = xtcks[1:length(panel.div.names)]+0.15, y = min(yl)-0.01, 
     labels = panel.div.names, xpd = NA, srt = 45, cex = 2, pos = 2)
title(xlab='Gene panel', line=14, cex.lab=fs)
title(ylab='Gene panel skew', line=6, cex.lab=fs)
dev.off()

png 
  2

### starmap

In [14]:
gs.names <- c('Fiber tracts', 'Ventricles', 'Dentate gyrus')
panel.div <- unlist(divs$starmap)[c(1:3,5)]
# panel.div.names <- c(gs.names[1:3], 'Random', 'Original')
panel.div.names <- c(gs.names, 'Original')
options(repr.plot.width=6, repr.plot.height=7)
pdf(paste0(save.dir, 'panel_div/starmap.pdf'),  
    width = 8, height = 9)
par(mfrow = c(1,1), mar = c(17.1, 10, 3.1, 2.1))
xtcks <- c(0:length(panel.div)+1)
x <- seq(length(panel.div))
y <- panel.div
xl <- c(0,max(x)+1)
yl <- c(0,0.14)
sp <- c(1.5, rep(1,length(panel.div)-1))

barplot(y, xlim = xl, ylim = yl, width = 0.5, space = sp,
    xaxt = "n", yaxt = "n", xaxs = 'i', yaxs = 'i',
    ylab = '', xlab = '')
axis(1, at = c(0,xtcks), cex.axis=2, labels = FALSE)
axis(2, at = seq(from = 0, to = ceiling(max(yl)), by = 0.02), cex.axis=3, las = 2)
text(x = xtcks[1:length(panel.div.names)]+0.15, y = min(yl)-0.01, 
     labels = panel.div.names, xpd = NA, srt = 45, cex = 2, pos = 2)
title(xlab='Gene panel', line=14, cex.lab=fs)
title(ylab='Gene panel skew', line=6, cex.lab=fs)
dev.off()

png 
  2

### seqfish

In [21]:
gs.names <- c('Cortex', 'Medulla', 'Pelvis')
panel.div <- unlist(divs$seqfish)[c(1:3,5)]
# panel.div.names <- c(gs.names[1:3], 'Random', 'Original')
panel.div.names <- c(gs.names, 'Original')
options(repr.plot.width=6, repr.plot.height=7)
pdf(paste0(save.dir, 'panel_div/seqfish.pdf'),  
    width = 8, height = 9)
par(mfrow = c(1,1), mar = c(17.1, 10, 3.1, 2.1))
xtcks <- c(0:length(panel.div)+1)
x <- seq(length(panel.div))
y <- panel.div
xl <- c(0,max(x)+1)
yl <- c(0,0.2)
sp <- c(1.5, rep(1,length(panel.div)-1))

barplot(y, xlim = xl, ylim = yl, width = 0.5, space = sp,
    xaxt = "n", yaxt = "n", xaxs = 'i', yaxs = 'i',
    ylab = '', xlab = '')
axis(1, at = c(0,xtcks), cex.axis=2, labels = FALSE)
axis(2, at = seq(from = 0, to = ceiling(max(yl)), by = 0.04), cex.axis=3, las = 2)
text(x = xtcks[1:length(panel.div.names)]+0.15, y = min(yl)-0.02, 
     labels = panel.div.names, xpd = NA, srt = 45, cex = 2, pos = 2)
title(xlab='Gene panel', line=14, cex.lab=fs)
title(ylab='Gene panel skew', line=6, cex.lab=fs)
dev.off()

png 
  2

### cosmx

In [16]:
divs$cosmx

$`Zone 3`
[1] 0.01636635

$`Zone 1`
[1] 0.02378937

$random
[1] 0.002756374

$full
[1] 0.00132335

In [18]:
gs.names <- c('Zone 3', 'Zone 1')
panel.div <- unlist(divs$cosmx)[c(1,2,4)]
# panel.div.names <- c(gs.names[1:2], 'Random', 'Original')
panel.div.names <- c(gs.names, 'Original')
options(repr.plot.width=6, repr.plot.height=7)
pdf(paste0(save.dir, 'panel_div/cosmx.pdf'),  
    width = 8, height = 9)
par(mfrow = c(1,1), mar = c(17.1, 10, 3.1, 2.1))
xtcks <- c(0:length(panel.div)+1)
x <- seq(length(panel.div))
y <- panel.div
xl <- c(0,max(x)+1)
yl <- c(0,0.03)
sp <- c(1.5, rep(1,length(panel.div)-1))

barplot(y, xlim = xl, ylim = yl, width = 0.5, space = sp,
    xaxt = "n", yaxt = "n", xaxs = 'i', yaxs = 'i',
    ylab = '', xlab = '')
axis(1, at = c(0,xtcks), cex.axis=2, labels = FALSE)
axis(2, at = seq(from = 0, to = ceiling(max(yl)), by = 0.01), cex.axis=3, las = 2)
text(x = xtcks[1:length(panel.div.names)]+0.15, y = min(yl)-0.005, 
     labels = panel.div.names, xpd = NA, srt = 45, cex = 2, pos = 2)
title(xlab='Gene panel', line=14, cex.lab=fs)
title(ylab='Gene panel skew', line=6, cex.lab=fs)
dev.off()

png 
  2

### xenium

In [20]:
gs.names <- c('Invasive', 'DCIS')
panel.div <- unlist(divs$xenium)[c(1,2,4)]
# panel.div.names <- c(gs.names[1:2], 'Random', 'Original')
panel.div.names <- c(gs.names, 'Original')
options(repr.plot.width=6, repr.plot.height=7)
pdf(paste0(save.dir, 'panel_div/xenium.pdf'),  
    width = 8, height = 9)
par(mfrow = c(1,1), mar = c(17.1, 10, 3.1, 2.1))
xtcks <- c(0:length(panel.div)+1)
x <- seq(length(panel.div))
y <- panel.div
xl <- c(0,max(x)+1)
yl <- c(0,0.05)
sp <- c(1.5, rep(1,length(panel.div)-1))

barplot(y, xlim = xl, ylim = yl, width = 0.5, space = sp,
    xaxt = "n", yaxt = "n", xaxs = 'i', yaxs = 'i',
    ylab = '', xlab = '')
axis(1, at = c(0,xtcks), cex.axis=2, labels = FALSE)
axis(2, at = seq(from = 0, to = ceiling(max(yl)), by = 0.01), cex.axis=3, las = 2)
text(x = xtcks[1:length(panel.div.names)]+0.15, y = min(yl)-0.005, 
     labels = panel.div.names, xpd = NA, srt = 45, cex = 2, pos = 2)
title(xlab='Gene panel', line=14, cex.lab=fs)
title(ylab='Gene panel skew', line=6, cex.lab=fs)
dev.off()

png 
  2

## sc size skew

In [88]:
sc.gssize.monocyte.divs <- lapply(sc.gssize.divs, function(x) {x[['monocytes']]})
max(unlist(sc.gssize.monocyte.divs))

[1] 1.126445

In [94]:
gs.names <- names(sc.gssize.monocyte.divs)
panel.div <- unlist(sc.gssize.monocyte.divs)
panel.div.names <- gs.names
options(repr.plot.width=6, repr.plot.height=7)
pdf(paste0(save.dir, 'panel_div/sc_size_monocytes.pdf'),  
    width = 8, height = 9)
par(mfrow = c(1,1), mar = c(17.1, 10, 3.1, 2.1))
xtcks <- c(0:length(panel.div)+1)
x <- seq(length(panel.div))
y <- panel.div
xl <- c(0,max(x)+1)
yl <- c(0,1.2)
sp <- c(1.5, rep(1,length(panel.div)-1))

barplot(y, xlim = xl, ylim = yl, width = 0.5, space = sp,
    xaxt = "n", yaxt = "n", xaxs = 'i', yaxs = 'i',
    ylab = '', xlab = '')
axis(1, at = c(0,xtcks), cex.axis=2, labels = FALSE)
axis(2, at = seq(from = 0, to = ceiling(max(yl)), by = 0.5), cex.axis=3, las = 2)
text(x = xtcks[1:length(panel.div.names)]+0.15, y = min(yl)-0.1, 
     labels = panel.div.names, xpd = NA, srt = 45, cex = 2, pos = 2)
title(xlab='Gene panel', line=14, cex.lab=fs)
title(ylab='Gene panel skew', line=6, cex.lab=fs)
dev.off()

png 
  2